# 디렉토리, 라이브러리, 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
import joblib
import matplotlib.pyplot as plt

from imblearn.over_sampling import BorderlineSMOTE

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### 디렉토리 설정 

In [ ]:
os.chdir('C:\\Users\\jy991\\바탕 화면\\2021 summer\\방학세미나')
os.getcwd()

### 데이터 불러오기

In [ ]:
train = pd.read_csv('train.csv')

# EDA

### 데이터 구조 확인

In [ ]:
train.info()

Columns = 201, Row = 28000

### 결측치 수 확인

In [ ]:
train.isnull().sum()

결측치가 없음을 확인!

### 데이터 불균형 확인

* 파이 그래프로 불균형 확인

In [ ]:
train_count = train['target'].value_counts()
train_count = pd.DataFrame(train_count)
train_count
plt.figure(figsize=(7,7))
plt.pie(train_count['target'], labels=train_count.index.values, autopct='%.1f%%',
       shadow=True,startangle=15,colors=(['#FADEE1','#BDDAE6']),explode=([0.2,0]),textprops={'fontsize': 20})

# 데이터 전처리

### Borderline_SMOTE

* 불균형 데이터 처리 전

In [ ]:
plt.bar(np.arange(2), train['target'].value_counts(), tick_label = ['0','1'],
        color='#CEA8BB',alpha=0.5)

* 데이터 타겟 변수로 나눠주기

In [ ]:
train_y = train['target']
train_x = train.drop(['target'], axis = 1)

* 불균형 데이터 처리 하기
* random_state=0으로 고정

In [ ]:
bsmote = BorderlineSMOTE(random_state = 916, kind = 'borderline-1')
X_res, y_res = bsmote.fit_resample(train_x, train_y) #Borderline_Smote 적용

train = pd.concat([X_res, y_res], axis = 1) #다시 하나의 데이터로 합쳐주기

plt.bar(np.arange(2), train['target'].value_counts(), tick_label = ['0','1'],
        color='#CEA8BB',alpha=0.5)

### PCA (Principal Component Analysis)

* 차원축소 (시간 복잡도 & 공간 복잡도를 줄이기 위해)

In [ ]:
train_y = train['target']
train_x = train.drop('target', axis = 1)

* 표준화 처리

In [ ]:
scale = StandardScaler().fit(train_x)
scaled_x = scale.transform(train_x)

* 95%의 분산 설명을 목표로 차원 축소
* n_components=0.95로 설정

In [ ]:
pca = PCA(n_components=0.95 , random_state = 0)
PCA = pca.fit(scaled_x)

* 줄어든 개수 확인

In [ ]:
features_pca = PCA.transform(scaled_x)

print("Original:", scaled_x.shape[1])
print("Final:", features_pca.shape[1])

* test set을 위한 모델 저장 (Scale & PCA)

In [ ]:
joblib.dump(scale, 'standard.pkl')
joblib.dump(PCA, 'pca.pkl')

* X변수와 y변수로 분리된 데이터를 다시 병합한 후 저장해주었습니다.

In [ ]:
train_x = pd.DataFrame(features_pca)
train = pd.concat([train_x,train_y],axis=1)

train.to_csv('train_pcaborder.csv', header = True, index = False)